# Implementing a new simulation code
Appendix C in the pyiron paper
https://www.sciencedirect.com/science/article/pii/S0927025618304786

In [1]:
class_lines = """\
from os.path import join
from pyiron.base.job.generic import GenericJob
from pyiron.base.generic.parameters import GenericParameters


class ToyJob(GenericJob):
    def __init__(self, project, job_name):
        super(ToyJob, self).__init__(project, job_name) 
        self.input = GenericParameters(table_name="input")  # Move this line to GenericJob 
        self.input['input_energy'] = 100
        # self.executable = "cat input > output"
        self._python_only_job = True
    
    # This function is executed 
    def run_static(self):
        with self.project_hdf5.open("output/generic") as h5out: 
             h5out["energy_tot"] = self.input["input_energy"]
        self.status.finished = True
    
    def to_hdf(self, hdf=None, group_name=None):  # Move this line to GenericJob 
        super(ToyJob, self).to_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.to_hdf(h5in)

    def from_hdf(self, hdf=None, group_name=None):  # Move this line to GenericJob 
        super(ToyJob, self).from_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.from_hdf(h5in)
"""

In [2]:
with open('toy.py', 'w') as t:
    t.writelines(class_lines)

In [3]:
from toy import ToyJob

In [4]:
from pyiron import Project

In [5]:
pr = Project('test')
pr.remove_jobs(recursive=True)

In [6]:
job = pr.create_job(job_type=ToyJob, job_name="toy")

In [7]:
job.run()

The job toy was saved and received the ID: 76


In [8]:
job['output/generic/energy_tot']

100

In [9]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,76,finished,None,toy,/toy,/Users/jan/pyiron/projects/,2020/2020-05-15-pyiron-meeting/test/,2020-05-15 14:53:18.971318,None,None,pyiron@MacBook-Pro.local#1,GenericJob,0.4,None,None


In [10]:
type(pr.load(job.job_name))

toy.ToyJob

In [11]:
type(pr.inspect(job.job_name))

pyiron.base.job.path.JobPath

In [12]:
type(pr.inspect(job.job_name).to_object())

toy.ToyJob